In [ ]:
# Preprocessing Channel Name

In [1]:
import re
import pandas as pd

dic = []
twitch_df = pd.read_csv("twitchdata-update.csv")

for idx, item in twitch_df.iterrows():
    if "(" in item["Channel"]:
        twitch_df["Channel"][idx] = re.findall("\(([a-z0-9_]+)\)", item["Channel"])[0]
    else:
        twitch_df["Channel"][idx] = item["Channel"]

In [ ]:
# Crawling

In [ ]:
import requests, json
from bs4 import BeautifulSoup

df = []
for channel in twitch_df["Channel"]:
    url = "https://twitchtracker.com/{}".format(channel)
    req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17'})
    soup = BeautifulSoup(req.text, 'html.parser')
    try:
        date = soup.select_one('#content-wrapper > div.container > section.row > div.col-md-3.col-sm-4 > ul > li:nth-child(5) > div:nth-child(2) > div:nth-child(2) > span').text
    except:
        date = None
    df.append({'Channel':channel, 'Date' : date})

date_df = pd.DataFrame(df)

In [ ]:
# Save as a csv file

In [ ]:
date_df.to_csv("twitch_date.csv")

In [ ]:
# merge into original dataset

In [4]:
twitch_date = pd.read_csv("twitch_date.csv")
twitch_date.drop(columns="Unnamed: 0",inplace=True)

In [8]:
twitch = twitch_df.merge(twitch_date, on='Channel')
twitch.tail(2)

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language,Date
998,AndyMilonakis,122311065,104745,10543,1153,547446,109111,3926918,True,False,English,2013-11-20 10:15
999,Remx,122192850,99180,13788,1205,178553,59432,2049420,True,False,French,2013-12-21 0:22


In [ ]:
# Save as a csv file

In [9]:
twitch.to_csv("twitch.csv")